# Weather Data Integration for Typhoon Impact Analysis

This notebook merges daily weather station data with typhoon impact records to create a comprehensive dataset for analysis.

## Objectives:
1. Load and understand both datasets
2. Clean and standardize data formats
3. Filter typhoons based on actual PAR entry/exit dates
4. Merge weather data with impact records based on nearest station and date
5. Extract EXTREME weather values only (strongest/worst conditions)
6. Handle missing data and validation
7. Export final merged dataset

## 1. Import Required Libraries

In [7]:
import pandas as pd
import numpy as np
import datetime as dt
from datetime import datetime, timedelta
import re
import warnings

# For distance calculations
from math import radians, cos, sin, asin, sqrt

# Display settings
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
warnings.filterwarnings('ignore')

print("Libraries imported successfully!")

Libraries imported successfully!


## 2. Load Datasets

In [8]:
# Load the datasets
print("Loading weather data...")
weather_data = pd.read_csv('../../data/Daily Data.csv')

print("Loading typhoon impact data...")
impact_data = pd.read_csv('../../data/data_withno_weatherdata.csv')

print("Loading typhoon duration/dates data...")
duration_data = pd.read_csv('../../data/Duration of typhoon.csv')

print(f"Weather data shape: {weather_data.shape}")
print(f"Impact data shape: {impact_data.shape}")
print(f"Duration data shape: {duration_data.shape}")
print("\nDatasets loaded successfully!")

Loading weather data...
Loading typhoon impact data...
Loading typhoon duration/dates data...
Weather data shape: (1828, 90)
Impact data shape: (1776, 26)
Duration data shape: (84, 7)

Datasets loaded successfully!


In [9]:
weather_data.head()

,Date(UTC),"Aparri, Cagayan Prec.Sum.Dly [mm]","Aparri, Cagayan Press.QFF.Min.Dly [hPa]","Aparri, Cagayan Wind.Speed.Dly [m/s]","Aparri, Cagayan Wind.Dir.Prevailing.Dly [deg.]","Baler (Radar), Aurora Prec.Sum.Dly [mm]","Baler (Radar), Aurora Press.QFF.Min.Dly [hPa]","Baler (Radar), Aurora Wind.Speed.Dly [m/s]","Baler (Radar), Aurora Wind.Dir.Prevailing.Dly [deg.]","Basco (Radar), Batanes Prec.Sum.Dly [mm]","Basco (Radar), Batanes Press.QFF.Min.Dly [hPa]","Basco (Radar), Batanes Wind.Speed.Dly [m/s]","Basco (Radar), Batanes Wind.Dir.Prevailing.Dly [deg.]","Borongan, Eastern Samar Prec.Sum.Dly [mm]","Borongan, Eastern Samar Press.QFF.Min.Dly [hPa]","Borongan, Eastern Samar Wind.Speed.Dly [m/s]","Borongan, Eastern Samar Wind.Dir.Prevailing.Dly [deg.]","Calayan, Cagayan Prec.Sum.Dly [mm]","Calayan, Cagayan Press.QFF.Min.Dly [hPa]","Calayan, Cagayan Wind.Speed.Dly [m/s]","Calayan, Cagayan Wind.Dir.Prevailing.Dly [deg.]","Casiguran, Aurora Prec.Sum.Dly [mm]","Casiguran, Aurora Press.QFF.Min.Dly [hPa]","Casiguran, Aurora Wind.Speed.Dly [m/s]","Casiguran, Aurora Wind.Dir.Prevailing.Dly [deg.]","Catarman, Northern Samar Prec.Sum.Dly [mm]","Catarman, Northern Samar Press.QFF.Min.Dly [hPa]","Catarman, Northern Samar Wind.Speed.Dly [m/s]","Catarman, Northern Samar Wind.Dir.Prevailing.Dly [deg.]","Catbalogan, Western Samar Prec.Sum.Dly [mm]","Catbalogan, Western Samar Press.QFF.Min.Dly [hPa]","Catbalogan, Western Samar Wind.Speed.Dly [m/s]","Catbalogan, Western Samar Wind.Dir.Prevailing.Dly [deg.]","Clark Airport (DMIA), Pampanga Prec.Sum.Dly [mm]","Clark Airport (DMIA), Pampanga Press.QFF.Min.Dly [hPa]","Clark Airport (DMIA), Pampanga Wind.Speed.Dly [m/s]","Clark Airport (DMIA), Pampanga Wind.Dir.Prevailing.Dly [deg.]","CLSU Muñoz, Nueva Ecija Prec.Sum.Dly [mm]","CLSU Muñoz, Nueva Ecija Press.QFF.Min.Dly [hPa]","CLSU Muñoz, Nueva Ecija Wind.Speed.Dly [m/s]","CLSU Muñoz, Nueva Ecija Wind.Dir.Prevailing.Dly [deg.]","Cubi Pt., Subic Bay Olongapo City Prec.Sum.Dly [mm]","Cubi Pt., Subic Bay Olongapo City Press.QFF.Min.Dly [hPa]","Cubi Pt., Subic Bay Olongapo City Wind.Speed.Dly [m/s]","Cubi Pt., Subic Bay Olongapo City Wind.Dir.Prevailing.Dly [deg.]","Daet, Camarines Norte Prec.Sum.Dly [mm]","Daet, Camarines Norte Press.QFF.Min.Dly [hPa]","Daet, Camarines Norte Wind.Speed.Dly [m/s]","Daet, Camarines Norte Wind.Dir.Prevailing.Dly [deg.]","Guiuan, Eastern Samar Prec.Sum.Dly [mm]","Guiuan, Eastern Samar Press.QFF.Min.Dly [hPa]","Guiuan, Eastern Samar Wind.Speed.Dly [m/s]","Guiuan, Eastern Samar Wind.Dir.Prevailing.Dly [deg.]","Iba, Zambales Prec.Sum.Dly [mm]","Iba, Zambales Press.QFF.Min.Dly [hPa]","Iba, Zambales Wind.Speed.Dly [m/s]","Iba, Zambales Wind.Dir.Prevailing.Dly [deg.]","Itbayat, Batanes Prec.Sum.Dly [mm]","Itbayat, Batanes Press.QFF.Min.Dly [hPa]","Itbayat, Batanes Wind.Speed.Dly [m/s]","Itbayat, Batanes Wind.Dir.Prevailing.Dly [deg.]","Juban, Sorsogon Prec.Sum.Dly [mm]","Juban, Sorsogon Press.QFF.Min.Dly [hPa]","Juban, Sorsogon Wind.Speed.Dly [m/s]","Juban, Sorsogon Wind.Dir.Prevailing.Dly [deg.]","Legazpi City, Albay Prec.Sum.Dly [mm]","Legazpi City, Albay Press.QFF.Min.Dly [hPa]","Legazpi City, Albay Wind.Speed.Dly [m/s]","Legazpi City, Albay Wind.Dir.Prevailing.Dly [deg.]","Maasin, Southern Leyte Prec.Sum.Dly [mm]","Maasin, Southern Leyte Press.QFF.Min.Dly [hPa]","Maasin, Southern Leyte Wind.Speed.Dly [m/s]","Maasin, Southern Leyte Wind.Dir.Prevailing.Dly [deg.]","Masbate City, Masbate Prec.Sum.Dly [mm]","Masbate City, Masbate Press.QFF.Min.Dly [hPa]","Masbate City, Masbate Wind.Speed.Dly [m/s]","Masbate City, Masbate Wind.Dir.Prevailing.Dly [deg.]","Tacloban City, Leyte Prec.Sum.Dly [mm]","Tacloban City, Leyte Press.QFF.Min.Dly [hPa]","Tacloban City, Leyte Wind.Speed.Dly [m/s]","Tacloban City, Leyte Wind.Dir.Prevailing.Dly [deg.]","Tuguegarao City, Cagayan Prec.Sum.Dly [mm]","Tuguegarao City, Cagayan Press.QFF.Min.Dly [hPa]","Tuguegarao City, Cagayan Wind.Speed.Dly [m/s]","Tuguegarao City, Cagayan Wind.Dir.Prevailing.Dly [deg.]"

In [10]:
weather_data.columns

Index(['Date(UTC)', 'Aparri, Cagayan Prec.Sum.Dly [mm]',
       'Aparri, Cagayan Press.QFF.Min.Dly [hPa]',
       'Aparri, Cagayan Wind.Speed.Dly [m/s]',
       'Aparri, Cagayan Wind.Dir.Prevailing.Dly [deg.]',
       'Baler (Radar), Aurora Prec.Sum.Dly [mm]',
       'Baler (Radar), Aurora Press.QFF.Min.Dly [hPa]',
       'Baler (Radar), Aurora Wind.Speed.Dly [m/s]',
       'Baler (Radar), Aurora Wind.Dir.Prevailing.Dly [deg.]',
       'Basco (Radar), Batanes Prec.Sum.Dly [mm]',
       'Basco (Radar), Batanes Press.QFF.Min.Dly [hPa]',
       'Basco (Radar), Batanes Wind.Speed.Dly [m/s]',
       'Basco (Radar), Batanes Wind.Dir.Prevailing.Dly [deg.]',
       'Borongan, Eastern Samar Prec.Sum.Dly [mm]',
       'Borongan, Eastern Samar Press.QFF.Min.Dly [hPa]',
       'Borongan, Eastern Samar Wind.Speed.Dly [m/s]',
       'Borongan, Eastern Samar Wind.Dir.Prevailing.Dly [deg.]',
       'Calayan, Cagayan Prec.Sum.Dly [mm]',
       'Calayan, Cagayan Press.QFF.Min.Dly [hPa]',
       'Calayan

In [11]:
data_without_weather.head()

,Typhoon Name,Year,Region,Province,City/Municipality,Families,Person,Brgy,Dead,Injured/Ill,Missing,Totally,Partially,Total,Quantity,Cost,Type,Category,Nearest_Station,Station_Province,Distance_km,Max_24hr_Rainfall_mm,Total_Storm_Rainfall_mm,Min_Pressure_hPa,Max_Sustained_Wind_kph,Duration_in_PAR_Hours
0,BETTY,2023,2,BATANES,BASCO,3608.0,11120.0,6.0,0.0,0.0,0.0,0.0,0.0,0.0,3608.0,2646179.36,['FAMILY FOOD PACK'],['FAMILY FOOD PACK'],BASCO,BATANES,2.497504,NaN,NaN,NaN,NaN,133.0
1,BETTY,2023,2,BATANES,ITBAYAT,968.0,3028.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,966.0,494592.00,['FAMILY FOOD PACK'],['FAMILY FOOD PACK'],ITBAYAT,BATANES,3.204943,NaN,NaN,NaN,NaN,133.0
2,BETTY,2023,2,BATANES,IVANA,444.0,1532.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,444.0,227328.00,['FAMILY FOOD PACK'],['FAMILY FOOD PACK'],BASCO,BATANES,9.470554,NaN,NaN,NaN,NaN,133.0
3,BETTY,2023,2,BATANES,MAHATAO,575.0,1792.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,575.0,291082.96,['FAMILY FOOD PACK'],['FAMILY FOOD PACK'],BASCO,BATANES,4.890816,NaN,NaN,NaN,NaN,133.0
4,BETTY,2023,2,BATANES,SABTANG,575.0,1955.0,6.0,0.0,0.0,0.0,0.0,0.0,0.0,575.0,296521.75,['FAMILY FOOD PACK'],['FAMILY FOOD PACK'],BASCO,BATANES,19.891231,NaN,NaN,NaN,NaN,133.0


In [12]:
data_without_weather.columns

Index(['Typhoon Name', 'Year', 'Region', 'Province', 'City/Municipality',
       'Families', 'Person', 'Brgy', 'Dead', 'Injured/Ill', 'Missing',
       'Totally', 'Partially', 'Total', 'Quantity', 'Cost', 'Type', 'Category',
       'Nearest_Station', 'Station_Province', 'Distance_km',
       'Max_24hr_Rainfall_mm', 'Total_Storm_Rainfall_mm', 'Min_Pressure_hPa',
       'Max_Sustained_Wind_kph', 'Duration_in_PAR_Hours'],
      dtype='object')

## 3. Understand Dataset Structure

In [13]:
print("=== WEATHER DATA STRUCTURE ===")
print(f"Shape: {weather_data.shape}")
print(f"Columns: {len(weather_data.columns)}")
print("\nFirst few columns:")
print(weather_data.columns[:10].tolist())
weather_data.head(3)

=== WEATHER DATA STRUCTURE ===
Shape: (1828, 90)
Columns: 90

First few columns:
['Date(UTC)', 'Aparri, Cagayan Prec.Sum.Dly [mm]', 'Aparri, Cagayan Press.QFF.Min.Dly [hPa]', 'Aparri, Cagayan Wind.Speed.Dly [m/s]', 'Aparri, Cagayan Wind.Dir.Prevailing.Dly [deg.]', 'Baler (Radar), Aurora Prec.Sum.Dly [mm]', 'Baler (Radar), Aurora Press.QFF.Min.Dly [hPa]', 'Baler (Radar), Aurora Wind.Speed.Dly [m/s]', 'Baler (Radar), Aurora Wind.Dir.Prevailing.Dly [deg.]', 'Basco (Radar), Batanes Prec.Sum.Dly [mm]']


,Date(UTC),"Aparri, Cagayan Prec.Sum.Dly [mm]","Aparri, Cagayan Press.QFF.Min.Dly [hPa]","Aparri, Cagayan Wind.Speed.Dly [m/s]","Aparri, Cagayan Wind.Dir.Prevailing.Dly [deg.]","Baler (Radar), Aurora Prec.Sum.Dly [mm]","Baler (Radar), Aurora Press.QFF.Min.Dly [hPa]","Baler (Radar), Aurora Wind.Speed.Dly [m/s]","Baler (Radar), Aurora Wind.Dir.Prevailing.Dly [deg.]","Basco (Radar), Batanes Prec.Sum.Dly [mm]","Basco (Radar), Batanes Press.QFF.Min.Dly [hPa]","Basco (Radar), Batanes Wind.Speed.Dly [m/s]","Basco (Radar), Batanes Wind.Dir.Prevailing.Dly [deg.]","Borongan, Eastern Samar Prec.Sum.Dly [mm]","Borongan, Eastern Samar Press.QFF.Min.Dly [hPa]","Borongan, Eastern Samar Wind.Speed.Dly [m/s]","Borongan, Eastern Samar Wind.Dir.Prevailing.Dly [deg.]","Calayan, Cagayan Prec.Sum.Dly [mm]","Calayan, Cagayan Press.QFF.Min.Dly [hPa]","Calayan, Cagayan Wind.Speed.Dly [m/s]","Calayan, Cagayan Wind.Dir.Prevailing.Dly [deg.]","Casiguran, Aurora Prec.Sum.Dly [mm]","Casiguran, Aurora Press.QFF.Min.Dly [hPa]","Casiguran, Aurora Wind.Speed.Dly [m/s]","Casiguran, Aurora Wind.Dir.Prevailing.Dly [deg.]","Catarman, Northern Samar Prec.Sum.Dly [mm]","Catarman, Northern Samar Press.QFF.Min.Dly [hPa]","Catarman, Northern Samar Wind.Speed.Dly [m/s]","Catarman, Northern Samar Wind.Dir.Prevailing.Dly [deg.]","Catbalogan, Western Samar Prec.Sum.Dly [mm]","Catbalogan, Western Samar Press.QFF.Min.Dly [hPa]","Catbalogan, Western Samar Wind.Speed.Dly [m/s]","Catbalogan, Western Samar Wind.Dir.Prevailing.Dly [deg.]","Clark Airport (DMIA), Pampanga Prec.Sum.Dly [mm]","Clark Airport (DMIA), Pampanga Press.QFF.Min.Dly [hPa]","Clark Airport (DMIA), Pampanga Wind.Speed.Dly [m/s]","Clark Airport (DMIA), Pampanga Wind.Dir.Prevailing.Dly [deg.]","CLSU Muñoz, Nueva Ecija Prec.Sum.Dly [mm]","CLSU Muñoz, Nueva Ecija Press.QFF.Min.Dly [hPa]","CLSU Muñoz, Nueva Ecija Wind.Speed.Dly [m/s]","CLSU Muñoz, Nueva Ecija Wind.Dir.Prevailing.Dly [deg.]","Cubi Pt., Subic Bay Olongapo City Prec.Sum.Dly [mm]","Cubi Pt., Subic Bay Olongapo City Press.QFF.Min.Dly [hPa]","Cubi Pt., Subic Bay Olongapo City Wind.Speed.Dly [m/s]","Cubi Pt., Subic Bay Olongapo City Wind.Dir.Prevailing.Dly [deg.]","Daet, Camarines Norte Prec.Sum.Dly [mm]","Daet, Camarines Norte Press.QFF.Min.Dly [hPa]","Daet, Camarines Norte Wind.Speed.Dly [m/s]","Daet, Camarines Norte Wind.Dir.Prevailing.Dly [deg.]","Guiuan, Eastern Samar Prec.Sum.Dly [mm]","Guiuan, Eastern Samar Press.QFF.Min.Dly [hPa]","Guiuan, Eastern Samar Wind.Speed.Dly [m/s]","Guiuan, Eastern Samar Wind.Dir.Prevailing.Dly [deg.]","Iba, Zambales Prec.Sum.Dly [mm]","Iba, Zambales Press.QFF.Min.Dly [hPa]","Iba, Zambales Wind.Speed.Dly [m/s]","Iba, Zambales Wind.Dir.Prevailing.Dly [deg.]","Itbayat, Batanes Prec.Sum.Dly [mm]","Itbayat, Batanes Press.QFF.Min.Dly [hPa]","Itbayat, Batanes Wind.Speed.Dly [m/s]","Itbayat, Batanes Wind.Dir.Prevailing.Dly [deg.]","Juban, Sorsogon Prec.Sum.Dly [mm]","Juban, Sorsogon Press.QFF.Min.Dly [hPa]","Juban, Sorsogon Wind.Speed.Dly [m/s]","Juban, Sorsogon Wind.Dir.Prevailing.Dly [deg.]","Legazpi City, Albay Prec.Sum.Dly [mm]","Legazpi City, Albay Press.QFF.Min.Dly [hPa]","Legazpi City, Albay Wind.Speed.Dly [m/s]","Legazpi City, Albay Wind.Dir.Prevailing.Dly [deg.]","Maasin, Southern Leyte Prec.Sum.Dly [mm]","Maasin, Southern Leyte Press.QFF.Min.Dly [hPa]","Maasin, Southern Leyte Wind.Speed.Dly [m/s]","Maasin, Southern Leyte Wind.Dir.Prevailing.Dly [deg.]","Masbate City, Masbate Prec.Sum.Dly [mm]","Masbate City, Masbate Press.QFF.Min.Dly [hPa]","Masbate City, Masbate Wind.Speed.Dly [m/s]","Masbate City, Masbate Wind.Dir.Prevailing.Dly [deg.]","Tacloban City, Leyte Prec.Sum.Dly [mm]","Tacloban City, Leyte Press.QFF.Min.Dly [hPa]","Tacloban City, Leyte Wind.Speed.Dly [m/s]","Tacloban City, Leyte Wind.Dir.Prevailing.Dly [deg.]","Tuguegarao City, Cagayan Prec.Sum.Dly [mm]","Tuguegarao City, Cagayan Press.QFF.Min.Dly [hPa]","Tuguegarao City, Cagayan Wind.Speed.Dly [m/s]","Tuguegarao City, Cagayan Wind.Dir.Prevailing.Dly [deg.]"

In [14]:
print("=== IMPACT DATA STRUCTURE ===")
print(f"Shape: {impact_data.shape}")
print(f"Columns: {len(impact_data.columns)}")
print("\nColumns:")
print(impact_data.columns.tolist())
impact_data.head(3)

=== IMPACT DATA STRUCTURE ===
Shape: (1776, 26)
Columns: 26

Columns:
['Typhoon Name', 'Year', 'Region', 'Province', 'City/Municipality', 'Families', 'Person', 'Brgy', 'Dead', 'Injured/Ill', 'Missing', 'Totally', 'Partially', 'Total', 'Quantity', 'Cost', 'Type', 'Category', 'Nearest_Station', 'Station_Province', 'Distance_km', 'Max_24hr_Rainfall_mm', 'Total_Storm_Rainfall_mm', 'Min_Pressure_hPa', 'Max_Sustained_Wind_kph', 'Duration_in_PAR_Hours']


,Typhoon Name,Year,Region,Province,City/Municipality,Families,Person,Brgy,Dead,Injured/Ill,Missing,Totally,Partially,Total,Quantity,Cost,Type,Category,Nearest_Station,Station_Province,Distance_km,Max_24hr_Rainfall_mm,Total_Storm_Rainfall_mm,Min_Pressure_hPa,Max_Sustained_Wind_kph,Duration_in_PAR_Hours
0,BETTY,2023,2,BATANES,BASCO,3608.0,11120.0,6.0,0.0,0.0,0.0,0.0,0.0,0.0,3608.0,2646179.36,['FAMILY FOOD PACK'],['FAMILY FOOD PACK'],BASCO,BATANES,2.497504,NaN,NaN,NaN,NaN,133.0
1,BETTY,2023,2,BATANES,ITBAYAT,968.0,3028.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,966.0,494592.00,['FAMILY FOOD PACK'],['FAMILY FOOD PACK'],ITBAYAT,BATANES,3.204943,NaN,NaN,NaN,NaN,133.0
2,BETTY,2023,2,BATANES,IVANA,444.0,1532.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,444.0,227328.00,['FAMILY FOOD PACK'],['FAMILY FOOD PACK'],BASCO,BATANES,9.470554,NaN,NaN,NaN,NaN,133.0


In [15]:
print("=== DURATION DATA STRUCTURE ===")
print(f"Shape: {duration_data.shape}")
print(f"Columns: {len(duration_data.columns)}")
print("\nColumns:")
print(duration_data.columns.tolist())
duration_data.head(10)

=== DURATION DATA STRUCTURE ===
Shape: (84, 7)
Columns: 7

Columns:
['YEAR', 'MONTH', 'TYPE', 'TC NAME', 'PAR BEG', 'PAR END', 'MSW']


,YEAR,MONTH,TYPE,TC NAME,PAR BEG,PAR END,MSW
0,2020,5,TY,AMBO,5/10/2020,5/17/2020,155
1,2020,6,TS,BUTCHOY,6/11/2020,6/12/2020,65
2,2020,7,TD,CARINA,7/13/2020,7/14/2020,45
3,2020,7,STS,DINDO,7/31/2020,8/3/2020,95
4,2020,8,TS,ENTENG,8/8/2020,8/9/2020,75
5,2020,8,TD,FERDIE,8/9/2020,8/10/2020,55
6,2020,8,TD,GENER,8/13/2020,8/13/2020,45
7,2020,8,TD,HELEN,8/17/2020,8/18/2020,45
8,2020,8,STS,IGME,8/21/2020,8/22/2020,100
9,2020,8,TY,JULIAN,8/28/2020,8/31/2020,150


In [16]:
# Check data types
print("=== WEATHER DATA INFO ===")
weather_data.info()

print("\n=== IMPACT DATA INFO ===")
impact_data.info()

print("\n=== DURATION DATA INFO ===")
duration_data.info()

=== WEATHER DATA INFO ===
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1828 entries, 0 to 1827
Data columns (total 90 columns):
 #   Column                                                            Non-Null Count  Dtype  
---  ------                                                            --------------  -----  
 0   Date(UTC)                                                         1828 non-null   object 
 1   Aparri, Cagayan Prec.Sum.Dly [mm]                                 1828 non-null   object 
 2   Aparri, Cagayan Press.QFF.Min.Dly [hPa]                           1828 non-null   object 
 3   Aparri, Cagayan Wind.Speed.Dly [m/s]                              1828 non-null   object 
 4   Aparri, Cagayan Wind.Dir.Prevailing.Dly [deg.]                    1828 non-null   object 
 5   Baler (Radar), Aurora Prec.Sum.Dly [mm]                           1826 non-null   object 
 6   Baler (Radar), Aurora Press.QFF.Min.Dly [hPa]                     1828 non-null   object 
 7   Baler (

## 4. Clean and Prepare Duration Data

In [17]:
# Clean and prepare duration data
print("Cleaning duration data...")
duration_clean = duration_data.copy()

# Convert dates
duration_clean['PAR_START'] = pd.to_datetime(duration_clean['PAR BEG'], format='%m/%d/%Y', errors='coerce')
duration_clean['PAR_END'] = pd.to_datetime(duration_clean['PAR END'], format='%m/%d/%Y', errors='coerce')

# Standardize typhoon names
duration_clean['TYPHOON_NAME_CLEAN'] = duration_clean['TC NAME'].str.strip().str.upper()

# Calculate duration in hours
duration_clean['DURATION_HOURS'] = (duration_clean['PAR_END'] - duration_clean['PAR_START']).dt.total_seconds() / 3600

# Remove invalid records
duration_clean = duration_clean.dropna(subset=['PAR_START', 'PAR_END'])

print(f"Valid duration records: {len(duration_clean)}")
print(f"Date range: {duration_clean['PAR_START'].min()} to {duration_clean['PAR_END'].max()}")

# Display unique typhoons
print(f"\nUnique typhoons in duration data: {len(duration_clean['TYPHOON_NAME_CLEAN'].unique())}")
print(sorted(duration_clean['TYPHOON_NAME_CLEAN'].unique()))

duration_clean[['YEAR', 'TYPHOON_NAME_CLEAN', 'PAR_START', 'PAR_END', 'DURATION_HOURS', 'MSW']].head(10)

Cleaning duration data...
Valid duration records: 84
Date range: 2020-05-10 00:00:00 to 2024-12-23 00:00:00

Unique typhoons in duration data: 69
['AGATON', 'AGHON', 'AMANG', 'AMBO', 'AURING', 'BASYANG', 'BETTY', 'BISING', 'BUTCHOY', 'CALOY', 'CARINA', 'CHEDENG', 'CRISING', 'DANTE', 'DINDO', 'DODONG', 'DOMENG', 'EGAY', 'EMONG', 'ENTENG', 'ESTER', 'FABIAN', 'FALCON', 'FERDIE', 'FLORITA', 'GARDO', 'GENER', 'GORING', 'GORIO', 'HANNA', 'HELEN', 'HENRY', 'HUANING', 'IGME', 'INDAY', 'INENG', 'ISANG', 'JENNY', 'JOLINA', 'JOSIE', 'JULIAN', 'KABAYAN', 'KARDING', 'KIKO', 'KRISTINE', 'LANNIE', 'LEON', 'LUIS', 'MARCE', 'MARING', 'MAYMAY', 'NANDO', 'NENENG', 'NIKA', 'OBET', 'ODETTE', 'OFEL', 'PAENG', 'PEPITO', 'QUEENIE', 'QUERUBIN', 'QUINTA', 'ROLLY', 'ROMINA', 'ROSAL', 'SIONY', 'TONYO', 'ULYSSES', 'VICKY']


,YEAR,TYPHOON_NAME_CLEAN,PAR_START,PAR_END,DURATION_HOURS,MSW
0,2020,AMBO,2020-05-10,2020-05-17,168.0,155
1,2020,BUTCHOY,2020-06-11,2020-06-12,24.0,65
2,2020,CARINA,2020-07-13,2020-07-14,24.0,45
3,2020,DINDO,2020-07-31,2020-08-03,72.0,95
4,2020,ENTENG,2020-08-08,2020-08-09,24.0,75
5,2020,FERDIE,2020-08-09,2020-08-10,24.0,55
6,2020,GENER,2020-08-13,2020-08-13,0.0,45
7,2020,HELEN,2020-08-17,2020-08-18,24.0,45
8,2020,IGME,2020-08-21,2020-08-22,24.0,100
9,2020,JULIAN,2020-08-28,2020-08-31,72.0,150


## 5. Filter Impact Data Based on Available Typhoons

In [18]:
# Clean impact data typhoon names
print("Filtering impact data based on available typhoons...")
impact_data_clean = impact_data.copy()

# Standardize typhoon names in impact data
impact_data_clean['Typhoon_Name_Clean'] = impact_data_clean['Typhoon Name'].str.strip().str.upper()

# Check which typhoons in impact data exist in duration data
available_typhoons = set(duration_clean['TYPHOON_NAME_CLEAN'].unique())
impact_typhoons = set(impact_data_clean['Typhoon_Name_Clean'].unique())

matched_typhoons = impact_typhoons.intersection(available_typhoons)
missing_typhoons = impact_typhoons - available_typhoons

print(f"\nTyphoons in impact data: {len(impact_typhoons)}")
print(f"Typhoons with duration data: {len(matched_typhoons)}")
print(f"Typhoons missing duration data: {len(missing_typhoons)}")

print(f"\nMissing typhoons: {sorted(missing_typhoons)}")

# Filter impact data to only include typhoons with duration data
impact_filtered = impact_data_clean[impact_data_clean['Typhoon_Name_Clean'].isin(matched_typhoons)].copy()

print(f"\nOriginal impact records: {len(impact_data_clean)}")
print(f"Filtered impact records: {len(impact_filtered)}")
print(f"Records removed: {len(impact_data_clean) - len(impact_filtered)}")

# Show distribution by year
print("\nFiltered data by year:")
print(impact_filtered['Year'].value_counts().sort_index())

Filtering impact data based on available typhoons...

Typhoons in impact data: 25
Typhoons with duration data: 25
Typhoons missing duration data: 0

Missing typhoons: []

Original impact records: 1776
Filtered impact records: 1776
Records removed: 0

Filtered data by year:
Year
2020    548
2021    387
2022    749
2023     92
Name: count, dtype: int64


## 6. Data Cleaning and Standardization

In [19]:
# Function to extract station names from weather data columns
def extract_station_names(weather_df):
    """Extract unique station names from weather data column names"""
    stations = set()
    for col in weather_df.columns:
        if col != 'Date(UTC)' and ',' in col:
            station = col.split(',')[0].strip()
            if station != 'Source':
                stations.add(station)
    return sorted(list(stations))

# Extract station names
weather_stations = extract_station_names(weather_data)
print(f"Weather stations found: {len(weather_stations)}")
print("\nStation names:")
for i, station in enumerate(weather_stations, 1):
    print(f"{i:2d}. {station}")

Weather stations found: 22

Station names:
 1. Aparri
 2. Baler (Radar)
 3. Basco (Radar)
 4. Borongan
 5. CLSU Muñoz
 6. Calayan
 7. Casiguran
 8. Catarman
 9. Catbalogan
10. Clark Airport (DMIA)
11. Cubi Pt.
12. Daet
13. Guiuan
14. Iba
15. Itbayat
16. Juban
17. Legazpi City
18. Maasin
19. Masbate City
20. Tacloban City
21. Tuguegarao City
22. Virac (Synop)


In [20]:
# Clean and standardize date column in weather data
print("Cleaning weather data dates...")
weather_data_clean = weather_data.copy()

# Convert date column
weather_data_clean['Date'] = pd.to_datetime(weather_data_clean['Date(UTC)'], errors='coerce')
weather_data_clean = weather_data_clean.dropna(subset=['Date'])

print(f"Weather data date range: {weather_data_clean['Date'].min()} to {weather_data_clean['Date'].max()}")
print(f"Valid weather records: {len(weather_data_clean)}")

Cleaning weather data dates...
Weather data date range: 2020-01-01 00:00:00 to 2024-12-31 00:00:00
Valid weather records: 1827


In [21]:
# Clean impact data stations
print("Cleaning impact station data...")

# Standardize station names
impact_filtered['Nearest_Station_Clean'] = impact_filtered['Nearest_Station'].str.strip().str.upper()

# Check unique stations in filtered impact data
impact_stations = impact_filtered['Nearest_Station_Clean'].unique()
print(f"\nUnique stations in filtered impact data: {len(impact_stations)}")
print(sorted(impact_stations))

Cleaning impact station data...

Unique stations in filtered impact data: 22
['APARRI', 'BALER', 'BASCO', 'BORONGAN', 'CABANATUAN', 'CALAYAN', 'CASIGURAN', 'CATARMAN', 'CATBALOGAN', 'CLARK AIRPORT', 'DAET', 'GUIUAN', 'IBA', 'ITBAYAT', 'JUBAN', 'LEGAZPI CITY', 'MAASIN', 'MASBATE', 'SUBIC BAY', 'TACLOBAN', 'TUGUEGARAO', 'VIRAC']


In [22]:
# Create station mapping between datasets
print("Creating station name mapping...")

# Standardize weather station names
weather_stations_clean = [station.strip().upper() for station in weather_stations]

# Find matching stations
matched_stations = []
unmatched_impact = []
unmatched_weather = []

for impact_station in impact_stations:
    found = False
    for weather_station in weather_stations_clean:
        if impact_station in weather_station or weather_station in impact_station:
            matched_stations.append((impact_station, weather_station))
            found = True
            break
    if not found:
        unmatched_impact.append(impact_station)

# Check for unmatched weather stations
matched_weather_stations = [match[1] for match in matched_stations]
for weather_station in weather_stations_clean:
    if weather_station not in matched_weather_stations:
        unmatched_weather.append(weather_station)

print(f"\nMatched stations: {len(matched_stations)}")
for match in matched_stations:
    print(f"  {match[0]} -> {match[1]}")

print(f"\nUnmatched impact stations: {len(unmatched_impact)}")
for station in unmatched_impact:
    print(f"  {station}")

print(f"\nUnmatched weather stations: {len(unmatched_weather)}")
for station in unmatched_weather:
    print(f"  {station}")

Creating station name mapping...

Matched stations: 20
  BASCO -> BASCO (RADAR)
  ITBAYAT -> ITBAYAT
  CALAYAN -> CALAYAN
  APARRI -> APARRI
  TUGUEGARAO -> TUGUEGARAO CITY
  CASIGURAN -> CASIGURAN
  BALER -> BALER (RADAR)
  CLARK AIRPORT -> CLARK AIRPORT (DMIA)
  IBA -> IBA
  LEGAZPI CITY -> LEGAZPI CITY
  DAET -> DAET
  VIRAC -> VIRAC (SYNOP)
  MASBATE -> MASBATE CITY
  JUBAN -> JUBAN
  CATBALOGAN -> CATBALOGAN
  TACLOBAN -> TACLOBAN CITY
  BORONGAN -> BORONGAN
  GUIUAN -> GUIUAN
  MAASIN -> MAASIN
  CATARMAN -> CATARMAN

Unmatched impact stations: 2
  CABANATUAN
  SUBIC BAY

Unmatched weather stations: 2
  CLSU MUÑOZ
  CUBI PT.


## 7. Create Typhoon Periods with Actual Dates

In [23]:
# Create typhoon periods using actual PAR entry/exit dates
print("Creating typhoon periods with actual dates...")

# Merge impact data with duration data to get actual dates
impact_with_dates = impact_filtered.merge(
    duration_clean[['YEAR', 'TYPHOON_NAME_CLEAN', 'PAR_START', 'PAR_END', 'DURATION_HOURS', 'MSW']],
    left_on=['Year', 'Typhoon_Name_Clean'],
    right_on=['YEAR', 'TYPHOON_NAME_CLEAN'],
    how='left'
)

print(f"Impact data with actual dates: {impact_with_dates.shape}")

# Check for missing dates
missing_dates = impact_with_dates['PAR_START'].isna().sum()
print(f"Records missing actual dates: {missing_dates}")

if missing_dates > 0:
    print("Records with missing dates:")
    missing_records = impact_with_dates[impact_with_dates['PAR_START'].isna()]
    print(missing_records[['Typhoon Name', 'Year', 'Typhoon_Name_Clean']].drop_duplicates())

# Remove records without valid dates
impact_with_dates = impact_with_dates.dropna(subset=['PAR_START', 'PAR_END'])
print(f"\nFinal records with valid dates: {len(impact_with_dates)}")

# Display sample of merged data
sample_cols = ['Typhoon Name', 'Year', 'Province', 'PAR_START', 'PAR_END', 'DURATION_HOURS', 'MSW']
print("\nSample of merged data:")
impact_with_dates[sample_cols].drop_duplicates(['Typhoon Name', 'Year']).head(10)

Creating typhoon periods with actual dates...
Impact data with actual dates: (1776, 34)
Records missing actual dates: 0

Final records with valid dates: 1776

Sample of merged data:
Impact data with actual dates: (1776, 34)
Records missing actual dates: 0

Final records with valid dates: 1776

Sample of merged data:


,Typhoon Name,Year,Province,PAR_START,PAR_END,DURATION_HOURS,MSW
0,BETTY,2023,BATANES,2023-05-27,2023-06-01,120.0,195
6,HENRY,2022,BATANES,2022-08-31,2022-09-04,96.0,195
7,JENNY,2023,BATANES,2023-09-29,2023-10-06,168.0,175
10,KIKO,2021,BATANES,2021-09-07,2021-09-12,120.0,215
16,MARING,2021,BATANES,2021-10-07,2021-10-12,120.0,100
22,FLORITA,2022,CAGAYAN,2022-08-21,2022-08-24,72.0,110
44,KARDING,2022,CAGAYAN,2022-09-22,2022-09-26,96.0,195
84,MAYMAY,2022,CAGAYAN,2022-10-10,2022-10-12,48.0,45
100,NENENG,2022,CAGAYAN,2022-10-13,2022-10-16,72.0,120
118,OBET,2022,CAGAYAN,2022-10-19,2022-10-22,72.0,55


## 8. Extract Weather Data by Station

In [24]:
def extract_weather_for_station(weather_df, station_name):
    """Extract weather data for a specific station"""
    station_cols = ['Date']
    weather_metrics = {}
    
    for col in weather_df.columns:
        if station_name in col:
            if 'Prec.Sum.Dly [mm]' in col:
                weather_metrics['rainfall_mm'] = col
            elif 'Press.QFF.Min.Dly [hPa]' in col:
                weather_metrics['pressure_hpa'] = col
            elif 'Wind.Speed.Dly [m/s]' in col:
                weather_metrics['wind_speed_ms'] = col
            elif 'Wind.Dir.Prevailing.Dly [deg.]' in col:
                weather_metrics['wind_direction_deg'] = col
    
    # Extract relevant columns
    extract_cols = ['Date'] + list(weather_metrics.values())
    station_data = weather_df[extract_cols].copy()
    
    # Rename columns to standard names
    rename_dict = {'Date': 'Date'}
    for metric, col in weather_metrics.items():
        rename_dict[col] = metric
    
    station_data = station_data.rename(columns=rename_dict)
    station_data['station'] = station_name
    
    return station_data

# Test extraction for one station
if weather_stations:
    test_station = weather_stations[0]
    test_data = extract_weather_for_station(weather_data_clean, test_station)
    print(f"Sample extraction for {test_station}:")
    print(test_data.head())
    print(f"\nColumns: {test_data.columns.tolist()}")

Sample extraction for Aparri:
        Date rainfall_mm pressure_hpa wind_speed_ms wind_direction_deg station
1 2020-01-01        0.00      1019.90           7.0               40.0  Aparri
2 2020-01-02        0.00      1018.70           6.0               60.0  Aparri
3 2020-01-03        0.00      1016.50           5.0               40.0  Aparri
4 2020-01-04        2.20      1015.70           5.0               40.0  Aparri
5 2020-01-05        0.00      1016.20           4.0               80.0  Aparri

Columns: ['Date', 'rainfall_mm', 'pressure_hpa', 'wind_speed_ms', 'wind_direction_deg', 'station']


In [25]:
# Create comprehensive weather dataset
print("Creating consolidated weather dataset...")

all_weather_data = []

for station in weather_stations:
    try:
        station_data = extract_weather_for_station(weather_data_clean, station)
        all_weather_data.append(station_data)
        print(f"Extracted {len(station_data)} records for {station}")
    except Exception as e:
        print(f"Error extracting data for {station}: {e}")

# Combine all station data
if all_weather_data:
    consolidated_weather = pd.concat(all_weather_data, ignore_index=True)
    print(f"\nConsolidated weather dataset: {consolidated_weather.shape}")
    print(f"Date range: {consolidated_weather['Date'].min()} to {consolidated_weather['Date'].max()}")
    print(f"Stations: {consolidated_weather['station'].nunique()}")
else:
    print("No weather data extracted!")

Creating consolidated weather dataset...
Extracted 1827 records for Aparri
Extracted 1827 records for Baler (Radar)
Extracted 1827 records for Basco (Radar)
Extracted 1827 records for Borongan
Extracted 1827 records for CLSU Muñoz
Extracted 1827 records for Calayan
Extracted 1827 records for Casiguran
Extracted 1827 records for Catarman
Extracted 1827 records for Catbalogan
Extracted 1827 records for Clark Airport (DMIA)
Extracted 1827 records for Cubi Pt.
Extracted 1827 records for Daet
Extracted 1827 records for Guiuan
Extracted 1827 records for Iba
Extracted 1827 records for Itbayat
Extracted 1827 records for Juban
Extracted 1827 records for Legazpi City
Extracted 1827 records for Maasin
Extracted 1827 records for Masbate City
Extracted 1827 records for Tacloban City
Extracted 1827 records for Tuguegarao City
Extracted 1827 records for Virac (Synop)

Consolidated weather dataset: (40194, 6)
Date range: 2020-01-01 00:00:00 to 2024-12-31 00:00:00
Stations: 22
Stations: 22


In [26]:
# Clean and standardize the consolidated weather data
print("Cleaning consolidated weather data...")

# Convert numeric columns
numeric_cols = ['rainfall_mm', 'pressure_hpa', 'wind_speed_ms', 'wind_direction_deg']
for col in numeric_cols:
    if col in consolidated_weather.columns:
        consolidated_weather[col] = pd.to_numeric(consolidated_weather[col], errors='coerce')

# Standardize station names
consolidated_weather['station_clean'] = consolidated_weather['station'].str.strip().str.upper()

# Remove rows where all weather metrics are missing
consolidated_weather = consolidated_weather.dropna(subset=['Date'])

print(f"Cleaned weather data: {consolidated_weather.shape}")
consolidated_weather.head()

Cleaning consolidated weather data...
Cleaned weather data: (40194, 7)
Cleaned weather data: (40194, 7)


,Date,rainfall_mm,pressure_hpa,wind_speed_ms,wind_direction_deg,station,station_clean
0,2020-01-01,0.0,1019.9,7.0,40.0,Aparri,APARRI
1,2020-01-02,0.0,1018.7,6.0,60.0,Aparri,APARRI
2,2020-01-03,0.0,1016.5,5.0,40.0,Aparri,APARRI
3,2020-01-04,2.2,1015.7,5.0,40.0,Aparri,APARRI
4,2020-01-05,0.0,1016.2,4.0,80.0,Aparri,APARRI


## 9. Extract EXTREME Weather Conditions During Typhoon Periods

In [27]:
# Function to get EXTREME weather statistics for typhoon period
def get_extreme_typhoon_weather(weather_df, station_name, start_date, end_date):
    """Get EXTREME weather statistics for a typhoon period - only the worst/strongest conditions"""
    
    # Filter weather data for station and date range
    station_weather = weather_df[
        (weather_df['station_clean'] == station_name) &
        (weather_df['Date'] >= start_date) &
        (weather_df['Date'] <= end_date)
    ]
    
    if station_weather.empty:
        return {
            'Extreme_24hr_Rainfall_mm': np.nan,
            'Total_Storm_Rainfall_mm': np.nan,
            'Lowest_Pressure_hPa': np.nan,
            'Strongest_Wind_kph': np.nan,
            'weather_records_found': 0,
            'typhoon_duration_days': 0
        }
    
    # Calculate EXTREME statistics - focusing on the most severe conditions
    duration_days = (end_date - start_date).days + 1
    
    stats = {
        'Extreme_24hr_Rainfall_mm': station_weather['rainfall_mm'].max() if 'rainfall_mm' in station_weather.columns else np.nan,
        'Total_Storm_Rainfall_mm': station_weather['rainfall_mm'].sum() if 'rainfall_mm' in station_weather.columns else np.nan,
        'Lowest_Pressure_hPa': station_weather['pressure_hpa'].min() if 'pressure_hpa' in station_weather.columns else np.nan,
        'Strongest_Wind_kph': station_weather['wind_speed_ms'].max() * 3.6 if 'wind_speed_ms' in station_weather.columns else np.nan,  # Convert m/s to km/h
        'weather_records_found': len(station_weather),
        'typhoon_duration_days': duration_days
    }
    
    return stats

# Test the function
if len(matched_stations) > 0:
    test_impact_station = matched_stations[0][0]
    test_weather_station = matched_stations[0][1]
    test_start = datetime(2020, 10, 19)  # Sample date
    test_end = datetime(2020, 10, 22)
    
    test_stats = get_extreme_typhoon_weather(
        consolidated_weather, test_weather_station, test_start, test_end
    )
    print(f"Test extreme weather stats for {test_weather_station}:")
    for key, value in test_stats.items():
        print(f"  {key}: {value}")

Test extreme weather stats for BASCO (RADAR):
  Extreme_24hr_Rainfall_mm: 45.9
  Total_Storm_Rainfall_mm: 128.5
  Lowest_Pressure_hPa: 1003.4
  Strongest_Wind_kph: 39.6
  weather_records_found: 4
  typhoon_duration_days: 4

  Extreme_24hr_Rainfall_mm: 45.9
  Total_Storm_Rainfall_mm: 128.5
  Lowest_Pressure_hPa: 1003.4
  Strongest_Wind_kph: 39.6
  weather_records_found: 4
  typhoon_duration_days: 4


## 10. Merge Weather Data with Impact Data

In [28]:
# Create station mapping dictionary for faster lookup
station_mapping = {impact: weather for impact, weather in matched_stations}
print(f"Station mapping created with {len(station_mapping)} entries")

# Initialize EXTREME weather columns in impact data
extreme_weather_cols = ['Extreme_24hr_Rainfall_mm', 'Total_Storm_Rainfall_mm', 'Lowest_Pressure_hPa', 'Strongest_Wind_kph']
for col in extreme_weather_cols:
    impact_with_dates[col] = np.nan

impact_with_dates['weather_records_found'] = 0
impact_with_dates['typhoon_duration_days'] = 0
impact_with_dates['weather_missing'] = 1

print("Initialized EXTREME weather columns in impact data")
print(f"Columns added: {extreme_weather_cols}")

Station mapping created with 20 entries
Initialized EXTREME weather columns in impact data
Columns added: ['Extreme_24hr_Rainfall_mm', 'Total_Storm_Rainfall_mm', 'Lowest_Pressure_hPa', 'Strongest_Wind_kph']


In [29]:
# Perform the weather data merge with EXTREME conditions
print("Merging EXTREME weather data with impact records...")
print("Note: Extracting only the most severe weather conditions during each typhoon period")

successful_merges = 0
failed_merges = 0
missing_stations = set()
missing_dates = []

# Process in batches for progress tracking
batch_size = 100
total_records = len(impact_with_dates)

for i in range(0, total_records, batch_size):
    batch_end = min(i + batch_size, total_records)
    batch = impact_with_dates.iloc[i:batch_end]
    
    for idx, row in batch.iterrows():
        impact_station = row['Nearest_Station_Clean']
        par_start = row['PAR_START']
        par_end = row['PAR_END']
        
        # Check if we have a mapping for this station
        if impact_station in station_mapping:
            weather_station = station_mapping[impact_station]
            
            # Skip if dates are missing
            if pd.isna(par_start) or pd.isna(par_end):
                failed_merges += 1
                missing_dates.append((row['Typhoon Name'], row['Year']))
                continue
            
            # Get EXTREME weather statistics
            extreme_stats = get_extreme_typhoon_weather(
                consolidated_weather, weather_station, par_start, par_end
            )
            
            # Update the impact data with extreme conditions
            for col, value in extreme_stats.items():
                if col in impact_with_dates.columns:
                    impact_with_dates.at[idx, col] = value
            
            if extreme_stats['weather_records_found'] > 0:
                impact_with_dates.at[idx, 'weather_missing'] = 0
                successful_merges += 1
            else:
                failed_merges += 1
        else:
            missing_stations.add(impact_station)
            failed_merges += 1
    
    # Progress update
    if (i // batch_size + 1) % 5 == 0:
        progress = (batch_end / total_records) * 100
        print(f"Progress: {progress:.1f}% ({batch_end}/{total_records} records)")

print(f"\nEXTREME weather merge complete!")
print(f"Successful merges: {successful_merges}")
print(f"Failed merges: {failed_merges}")
print(f"Success rate: {(successful_merges / total_records) * 100:.1f}%")

Merging EXTREME weather data with impact records...
Note: Extracting only the most severe weather conditions during each typhoon period
Progress: 28.2% (500/1776 records)
Progress: 28.2% (500/1776 records)
Progress: 56.3% (1000/1776 records)
Progress: 56.3% (1000/1776 records)
Progress: 84.5% (1500/1776 records)
Progress: 84.5% (1500/1776 records)

EXTREME weather merge complete!
Successful merges: 1613
Failed merges: 163
Success rate: 90.8%

EXTREME weather merge complete!
Successful merges: 1613
Failed merges: 163
Success rate: 90.8%


## 11. Analyze Extreme Weather Data

In [30]:
# Analyze extreme weather patterns
print("=== EXTREME WEATHER DATA ANALYSIS ===")

missing_weather_count = impact_with_dates['weather_missing'].sum()
total_records = len(impact_with_dates)

print(f"Total records: {total_records}")
print(f"Records with missing weather: {missing_weather_count}")
print(f"Records with EXTREME weather data: {total_records - missing_weather_count}")
print(f"EXTREME weather coverage: {((total_records - missing_weather_count) / total_records) * 100:.1f}%")

print(f"\nMissing stations ({len(missing_stations)}):")
for station in sorted(missing_stations):
    count = impact_with_dates[impact_with_dates['Nearest_Station_Clean'] == station].shape[0]
    print(f"  {station}: {count} records")

if missing_dates:
    print(f"\nMissing date periods ({len(set(missing_dates))}):")
    for typhoon, year in sorted(set(missing_dates)):
        print(f"  {typhoon} {year}")

=== EXTREME WEATHER DATA ANALYSIS ===
Total records: 1776
Records with missing weather: 163
Records with EXTREME weather data: 1613
EXTREME weather coverage: 90.8%

Missing stations (2):
  CABANATUAN: 83 records
  SUBIC BAY: 80 records


In [31]:
# Summary statistics for EXTREME weather data
print("=== EXTREME WEATHER DATA SUMMARY ===")

extreme_summary = impact_with_dates[extreme_weather_cols].describe()
print(extreme_summary)

# Check for any successful matches
has_weather = impact_with_dates['weather_missing'] == 0
if has_weather.sum() > 0:
    print(f"\n=== SUCCESSFULLY MERGED RECORDS WITH EXTREME CONDITIONS ===")
    sample_merged = impact_with_dates[has_weather].head()
    display_cols = ['Typhoon Name', 'Year', 'Province', 'Nearest_Station', 'PAR_START', 'PAR_END'] + extreme_weather_cols
    print(sample_merged[display_cols])
    
    # Show some statistics about extreme conditions
    print(f"\n=== EXTREME CONDITIONS STATISTICS ===")
    valid_weather = impact_with_dates[has_weather]
    
    if not valid_weather['Extreme_24hr_Rainfall_mm'].isna().all():
        max_rainfall = valid_weather['Extreme_24hr_Rainfall_mm'].max()
        print(f"Highest 24hr rainfall recorded: {max_rainfall:.1f} mm")
    
    if not valid_weather['Lowest_Pressure_hPa'].isna().all():
        min_pressure = valid_weather['Lowest_Pressure_hPa'].min()
        print(f"Lowest pressure recorded: {min_pressure:.1f} hPa")
    
    if not valid_weather['Strongest_Wind_kph'].isna().all():
        max_wind = valid_weather['Strongest_Wind_kph'].max()
        print(f"Strongest wind recorded: {max_wind:.1f} km/h")
else:
    print("\nNo records successfully merged with EXTREME weather data.")

=== EXTREME WEATHER DATA SUMMARY ===
       Extreme_24hr_Rainfall_mm  Total_Storm_Rainfall_mm  Lowest_Pressure_hPa  \
count               1613.000000              1613.000000          1612.000000   
mean                 105.303125               169.334619           995.586104   
std                   71.217412               105.491013            10.315003   
min                    0.000000                 0.000000           924.900000   
25%                   50.000000                92.700000           991.700000   
50%                  102.100000               154.610000           998.100000   
75%                  138.400000               234.500000          1001.850000   
max                  510.000000               770.500000          1008.600000   

       Strongest_Wind_kph  
count         1613.000000  
mean            18.247737  
std              8.930213  
min              3.600000  
25%             14.400000  
50%             14.400000  
75%             21.600000  
max      

## 12. Final Dataset Preparation and Export

In [32]:
# Prepare final dataset
print("Preparing final dataset with EXTREME weather conditions...")

final_dataset = impact_with_dates.copy()

# Remove temporary columns
columns_to_remove = ['Nearest_Station_Clean', 'Typhoon_Name_Clean', 'YEAR', 'TYPHOON_NAME_CLEAN']
final_dataset = final_dataset.drop(columns=[col for col in columns_to_remove if col in final_dataset.columns])

# Reorder columns to put weather data after existing columns
original_cols = [col for col in impact_data.columns if col in final_dataset.columns]
new_cols = ['PAR_START', 'PAR_END', 'DURATION_HOURS', 'MSW', 'weather_records_found', 'typhoon_duration_days', 'weather_missing'] + extreme_weather_cols
final_cols = original_cols + [col for col in new_cols if col not in original_cols]

final_dataset = final_dataset[final_cols]

print(f"Final dataset shape: {final_dataset.shape}")
print(f"Final columns ({len(final_dataset.columns)}): {final_dataset.columns.tolist()}")

Preparing final dataset with EXTREME weather conditions...
Final dataset shape: (1776, 36)
Final columns (36): ['Typhoon Name', 'Year', 'Region', 'Province', 'City/Municipality', 'Families', 'Person', 'Brgy', 'Dead', 'Injured/Ill', 'Missing', 'Totally', 'Partially', 'Total', 'Quantity', 'Cost', 'Type', 'Category', 'Nearest_Station', 'Station_Province', 'Distance_km', 'Max_24hr_Rainfall_mm', 'Total_Storm_Rainfall_mm', 'Min_Pressure_hPa', 'Max_Sustained_Wind_kph', 'Duration_in_PAR_Hours', 'PAR_START', 'PAR_END', 'DURATION_HOURS', 'MSW', 'weather_records_found', 'typhoon_duration_days', 'weather_missing', 'Extreme_24hr_Rainfall_mm', 'Lowest_Pressure_hPa', 'Strongest_Wind_kph']


In [33]:
# Export the merged dataset
output_file = '../../data/merged_extreme_weather_data.csv'
print(f"Exporting merged EXTREME weather dataset to {output_file}...")

try:
    final_dataset.to_csv(output_file, index=False)
    print(f"Successfully exported {len(final_dataset)} records to {output_file}")
except Exception as e:
    print(f"Error exporting data: {e}")
    # Try alternative location
    alt_output = 'merged_extreme_weather_data.csv'
    final_dataset.to_csv(alt_output, index=False)
    print(f"Exported to alternative location: {alt_output}")

Exporting merged EXTREME weather dataset to ../../data/merged_extreme_weather_data.csv...
Successfully exported 1776 records to ../../data/merged_extreme_weather_data.csv


## 13. Final Summary and Validation

In [34]:
# Final summary report
print("="*70)
print("           FINAL EXTREME WEATHER MERGE SUMMARY")
print("="*70)

print(f"Original impact records: {len(impact_data):,}")
print(f"Records with valid typhoons: {len(impact_filtered):,}")
print(f"Final merged records: {len(final_dataset):,}")
print(f"Weather stations available: {len(weather_stations)}")
print(f"Stations successfully mapped: {len(matched_stations)}")

extreme_coverage = ((len(final_dataset) - final_dataset['weather_missing'].sum()) / len(final_dataset)) * 100
print(f"EXTREME weather data coverage: {extreme_coverage:.1f}%")

print(f"\nEXTREME weather metrics added:")
for col in extreme_weather_cols:
    non_null = final_dataset[col].notna().sum()
    coverage = (non_null / len(final_dataset)) * 100
    print(f"  {col}: {non_null:,} records ({coverage:.1f}%)")

# Show typhoon classification distribution
print(f"\nTyphoon intensity distribution:")
if 'MSW' in final_dataset.columns:
    final_dataset['Typhoon_Category'] = final_dataset['MSW'].apply(
        lambda x: 'TD (≤61)' if x <= 61 else
                  'TS (62-88)' if x <= 88 else
                  'STS (89-117)' if x <= 117 else
                  'TY (118-184)' if x <= 184 else
                  'STY (>184)' if x > 184 else 'Unknown'
    )
    print(final_dataset['Typhoon_Category'].value_counts())

print(f"\nOutput file: merged_extreme_weather_data.csv")
print(f"File size: {final_dataset.memory_usage(deep=True).sum() / 1024 / 1024:.1f} MB")

print("\n" + "="*70)
print("EXTREME weather merge process completed successfully!")
print("Dataset now contains only the most severe weather conditions")
print("during each typhoon's presence in Philippine waters.")
print("="*70)

           FINAL EXTREME WEATHER MERGE SUMMARY
Original impact records: 1,776
Records with valid typhoons: 1,776
Final merged records: 1,776
Weather stations available: 22
Stations successfully mapped: 20
EXTREME weather data coverage: 90.8%

EXTREME weather metrics added:
  Extreme_24hr_Rainfall_mm: 1,613 records (90.8%)
  Total_Storm_Rainfall_mm: 1,613 records (90.8%)
  Lowest_Pressure_hPa: 1,612 records (90.8%)
  Strongest_Wind_kph: 1,613 records (90.8%)

Typhoon intensity distribution:
Typhoon_Category
STY (>184)      504
STS (89-117)    503
TY (118-184)    468
TS (62-88)      234
TD (≤61)         67
Name: count, dtype: int64

Output file: merged_extreme_weather_data.csv
File size: 1.2 MB

EXTREME weather merge process completed successfully!
Dataset now contains only the most severe weather conditions
during each typhoon's presence in Philippine waters.

           FINAL EXTREME WEATHER MERGE SUMMARY
Original impact records: 1,776
Records with valid typhoons: 1,776
Final merged re

In [35]:
# Display sample of final merged data with extreme conditions
print("Sample of final merged dataset with EXTREME weather conditions:")
sample_cols = ['Typhoon Name', 'Year', 'Province', 'City/Municipality', 'Dead', 'Injured/Ill', 'Families', 
              'PAR_START', 'PAR_END', 'MSW', 'Extreme_24hr_Rainfall_mm', 'Lowest_Pressure_hPa', 
              'Strongest_Wind_kph', 'weather_missing']
available_cols = [col for col in sample_cols if col in final_dataset.columns]

# Show records with weather data first
has_weather_sample = final_dataset[final_dataset['weather_missing'] == 0]
if len(has_weather_sample) > 0:
    print("\nRecords with EXTREME weather data:")
    print(has_weather_sample[available_cols].head(10))
else:
    print("\nAll records (no weather data available):")
    print(final_dataset[available_cols].head(10))

Sample of final merged dataset with EXTREME weather conditions:

Records with EXTREME weather data:
  Typhoon Name  Year Province City/Municipality  Dead  Injured/Ill  Families  \
0        BETTY  2023  BATANES             BASCO   0.0          0.0    3608.0   
1        BETTY  2023  BATANES           ITBAYAT   0.0          0.0     968.0   
2        BETTY  2023  BATANES             IVANA   0.0          0.0     444.0   
3        BETTY  2023  BATANES           MAHATAO   0.0          0.0     575.0   
4        BETTY  2023  BATANES           SABTANG   0.0          0.0     575.0   
5        BETTY  2023  BATANES            UYUGAN   0.0          0.0     433.0   
6        HENRY  2022  BATANES             BASCO   0.0          0.0       2.0   
7        JENNY  2023  BATANES             BASCO   0.0          0.0       1.0   
8        JENNY  2023  BATANES           ITBAYAT   0.0          0.0       6.0   
9        JENNY  2023  BATANES           SABTANG   0.0          0.0       0.0   

   PAR_START    PAR